In [ ]:
!pip install selenium

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks'
!pip install chromedriver-autoinstaller

In [ ]:
import os
import re
import time
import random
import numpy as np
import pandas as pd
import warnings; warnings.filterwarnings(action='ignore')

from tqdm import tqdm

# beautifulsoup
from bs4 import BeautifulSoup as bs

# selenium
from selenium import webdriver as wd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

# selenium options
options = wd.ChromeOptions()
options.add_argument('disable-gup')
options.add_argument('lang=ko_KR')
options.add_argument('--start-maximized')
options.add_argument("user-agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")

In [ ]:
# chrome driver
chromedriver_path = "./chromedriver_v125.exe"
service = Service(executable_path=chromedriver_path)
driver = wd.Chrome(service=service, options=options)

In [ ]:
# URL
wait_sec = random.uniform(1.5, 3.0)
url = 'https://www.musinsa.com/categories/item/001001?d_cat_cd=001001&brand=musinsastandard&list_kind=small&sort=pop_category&sub_sort=&page=1&display_cnt=90&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure='
driver.get(url)
time.sleep(wait_sec)

In [ ]:
# Crawling
# Step1 - get links
pages = driver.find_elements(By.CSS_SELECTOR, "div.wrapper > a.paging-btn.btn")

next_page_link_list = []
for page in pages:
    next_page_link_list.append(page.get_attribute("onclick"))

next_page_link_list = list(set(next_page_link_list))[:-1]

In [ ]:
# Step2 - get obj links
def get_obj_links():
    html = driver.page_source
    soup = bs(html, 'html.parser')

    obj_link = []
    objs = soup.select("#searchList > li.li_box")

    for obj in objs:
        obj_link.append("https:" + obj.find('a', attrs={'name': 'goods_link'}).get("href"))

    return obj_link

In [ ]:
total_obj_link = []

# first page
total_obj_link.append(get_obj_links())

# rest of the pages
for nex_page in next_page_link_list:
    driver.execute_script(nex_page)
    time.sleep(wait_sec)

    total_obj_link.append(get_obj_links())

# make one-dimensional list
total_obj_link = [item for sublist in total_obj_link for item in sublist]

In [ ]:
# Step3 - get review info of links
def get_review_info(num_page):
    review_info_list = []

    for num in tqdm(range(2, int(num_page) + 1)):
        driver.execute_script(f"ReviewPage.goPage({num}); return false;")
        time.sleep(wait_sec)

        html = driver.page_source
        soup = bs(html, 'html.parser')

        review_list = soup.select("div#reviewListFragment > div.review-list")

        for review in review_list:
            try:
                body_info = review.select_one("p.review-profile__body_information").text
            except:
                body_info = ""

            try:
                goods = review.select_one("span.review-goods-information__option").text.replace("\n", "").strip()
            except:
                goods = ""

            try:
                eval = review.select_one("ul.review-evaluation--type3__list").text.replace("\n", " ").strip()
            except:
                eval = ""

            review_info = {"body_info": body_info,
                           "goods": goods,
                           "eval": eval}

            review_info_list.append(review_info)

    return review_info_list

In [ ]:
total_review_info_list = []

for obj in total_obj_link:
    driver.get(obj)
    time.sleep(wait_sec)

    # num of review pages
    try:
        num_page = driver.find_element(By.CSS_SELECTOR, "div.box_page_msg").text.split(" ")[0]
    except:
        print("There are no reviews")
        continue

    review_info_list = get_review_info(num_page)
    total_review_info_list.append(review_info_list)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.89s/it]


There are no reviews


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.91s/it]


In [ ]:
# make one-dimensional list
total_review_info_list = [item for sublist in total_review_info_list for item in sublist]

In [ ]:
# make DataFrame
df = pd.DataFrame(total_review_info_list)